# Heim and Kratzer partiality

**The main question**: How can an analysis of presuppositions and presupposition projection be implemented?
 * There is very little literature on this, and only a couple of DRT implementations that I have been able to find.  The most comprehensive discussion is Johan Bos, *Implementing the Binding and Accommodation Theory for Anaphora Resolution and Presupposition Projection*, Computational Linguistics 29, 2003.
 http://cognet.mit.edu/journal/10.1162/089120103322145306
 * Other DRT work
 * Nothing that I've found for any other approaches.

## Background: a non-partial compositional semantics

The composition system in H&K chapter 3:

1. *Terminal nodes* (TN)  
If $\alpha$ is a terminal node, $[[\alpha]]$ is specified in the lexicon.

2. *Non-branching nodes* (NN)  
If $\alpha$ is a non-branching node, and $\beta$ is its daughter node, then $[[\alpha]] = [[\beta]]$.

3. *Functional application* (FA)  
If $\alpha$ is a branching node, $\{\beta, \gamma\}$ is the set of $\alpha$'s daughters, and $[[\beta]]$ is a function whose domain contains $[[\gamma]]$, then $[[\alpha]] = [[\beta]]([[\gamma]])$

All functions are assumed to be total.

## A partial semantics for presuppositions

Strawson's intuition: sentences with presupposition failures are neither true nor false.

    (1) The king of france is bald.

How to implement trivalence?

* Option A: a trivalent logic of some kind for the metalanguage.
* Option B: reference failure in the interpretive process (somehow).

The Heim & Kratzer solution is, on the surface, a version of option B: treat denotations as (potentially) partial functions, and adjust composition rules for partiality.

1. *Functional application (partial)* (H&K p. 76)  
If $\alpha$ is a branching node and $\{\beta, \gamma\}$ is the set of $\alpha$'s daughters, then $\alpha$ is in the domain of $[[ ]]$ if both $\beta$ and $\gamma$ are, and $[[\gamma]]$ is in the domain of $[[\beta]]$.  In this case, $[[\alpha]] = [[\beta]]([[\gamma]])$.

2. Principle of interpretability  
A constituent XP is interpretable only if XP is in the domain of $[[ ]]$.


**Some questions**:
* How exactly to track partiality conditions during composition?
* Partiality appears to be mainly a property of denotations.  Can an implementation work by lifting the composition rules, and simply adding definedness conditions to functions?
* How might this interact with the lack of a well-defined metalanguage in H&K?
* How seamless is the integration with other machinery, e.g. quantifiers?

## Metalanguage strategy 0: minimal modification of the metalanguage

Suppose that we have the following two H&K-style entries:

* $[[the]] = \lambda f : f \in D_{<e,t>} \text{ and there is exactly one }x\text{ such that }f(x)=1 . \text{the unique }x\text{ such that }f(x)=1$
* $[[cat]] = \lambda x : x \in D_e . x\text{ is a cat}$

What we need to check is if $[[cat]]$ is in the domain of $[[the]]$.  This is quite straightforward -- the definedness condition tells us that it is just in case we can find exactly one cat.  This looks good for the minimal strategy which involves:

1. When you apply FA on $f$ and $a$:
  
  1. Check if the argument satisfies the definedness condition of the function. If it doesn't, fail.
  2. If it does, return the result of $f(a)$.
  
At first glance, it appears that this can be done without much change to the metalanguage implementation!

### Complications
  
**Complication 1: arbitrary embedding of partiality.** However, in the general case, things may be more complicated.  What if we have an argument with a definedness condition as well?

* $[[cat]] = \lambda x : x \in D_e \text{ and } x\text{ is alive} . x\text{ is a cat}$  

Or in more logical notation:

* $[[cat]] = \lambda x_e : \text{Alive}(x) . \text{Cat}(x)$  
* $[[the]] = \lambda f_{<e,t>} : \exists ! x_e : f(x) . \iota x_e : f(x)$

Implementing step A above is no longer simple. First reduction step:

$[[the]]([[cat]]) = \exists ! x_e : [\lambda x_e : \text{Alive}(x) . \text{Cat}(x)](x) . \iota x_e : [\lambda x_e : \text{Alive}(x) . \text{Cat}(x)](x)$

 * To check if this [[cat]] is in the domain of [[the]], you have to check if $x$ is in the domain of [[cat]].
 * $x$ is bound by a quantifier in the presupposition, so you have to have a story about how this interacts.
 * $x$ is also bound by an operator in the body.
 * Whatever happens to the partiality condition, it has to be carried by non-functions *in the metalanguage*.  At a minimum, operators will have to deal with it somehow.
 * But also, instances of application in the metalanguage that may involve partiality can be *arbitrarily embedded* in formulas. Using a non-logical metalanguage obscures this problem, but doesn't eliminate it.

What even is the right result for the second and third reduction steps??

**Complication 2: projecting partiality.**

Suppose you have the following two expressions:

1. $[[A]] = \lambda f_{<e,t>} . \lambda x_e . P(x) \wedge f(x)$
2. $[[B]] = \lambda x_e : Q(x) . R(x)$

The first reduction step should look something like:

* $\lambda x_e . P(x) \wedge [\lambda x_e : Q(x) . R(x)](x)$

This instantiates the problem from above: partiality is happening at an embedded position in the formula.  But it also instantiates a new problem: when a human is working with H&K partial functions, they are actually doing a lot of conversion/simplification implicitly.  It's clear in this case what the result should be mathematically:

* $\lambda x_e : Q(x) . P(x) \wedge R(x)$

But how to get there?  Effectively what this requires is a precise algorithm for projecting partiality in metalanguage formulas.

**Complication 3: binding into partiality**

This is already instantiated by the above cases, but is useful to point out on its own terms. In general the metalanguage *cannot avoid* handling binding into partiality.  For example:

1. $[[A]] = \lambda f_{<e,t>} . \exists x_e . P(x) \wedge f(x)$
2. $[[B]] = \lambda x_e : Q(x) . R(x)$

First reduction:

* $\exists x_e . P(x) \wedge [\lambda x_e : Q(x) . R(x)](x)$

Whatever the result of this reduction is, $x$ to a first approximation is bound in both the definedness condition, and the regular part of an embedded formula.  In this case it *isn't* entirely obvious what the result should be, but something like:

* $\exists x_e . P(x) \wedge R(x)$, defined iff $\exists x_e . Q(x)$

looks plausible.

## Metalanguage strategy 1: multidimensional types

See separate notebook.

## Metalanguage strategy 2: a partiality operator outside the type system

One fundamental element of the Heim & Kratzer treatment is that partiality doesn't really change types.
* technical change: $D_{<\alpha, \beta>}$ is the set of all *partial* functions from $D_\alpha$ to $D_\beta$.

But we have seen that *partiality is not just about functions*.  The lambda notebook implementation of functions rests in the `LFun` class, along with machinery for doing reduction.  But we need a more general way of implementing partiality.

Basic reminder of how the lambda notebook implementation is structured.
* Each type constructor corresponds to a python class that implements a representation and set of behaviors for that type constructor.
* E.g. `LFun` stores a (typed) variable and a body, which is a typed expression, and defines reudction behaviors.  `TypedTerm` stores a term name and a type.  `Tuple` stores a sequence of typed expressions at arbitrary types.
* The minimal implementation suggested that one could simply create a variant of `LFun` that has a third part to its representation -- a definedness condition.  However, the points above force the conclusion that this isn't general enough.

*Basic idea*: add a new kind of typed expression, called `Partial`.  `Partial` objects:
 * Store a pair consisting of an arbitrary typed expression, and an object at type $t$.
 * Inherit their type from the first element in this pair.

All typed expressions add:
 * an algorithm for calculating the projection of partiality.  This isn't intended to be a good presupposition projection algorithm, but rather, to be a good theory of partiality.
 
**The algorithm `CP`**. For some formula $\alpha$ with subformulas $\beta_1...\beta_n$ for $n \geq 1$:
1. If $\alpha$ is a `Partial`, $CP(\alpha)$ is $Partial(Partial.body \wedge CP(Partial.body).body, Partial.condition \wedge CP(Partial.body).condition \wedge CP(Partial.condition).body \wedge CP(Partial.condition).condition)$
2. If $\alpha$ is an `LFun` $\lambda x . \phi$, $CP(\alpha)$ is $\lambda x . CP(\phi)$.
3. If $\alpha$ is a non-`LFun` binding expression, the details vary, though they all start by applying CP to the body.
  1. $\exists x : \beta$: $Partial(\exists x : CP(\beta).body, \exists x : CP(\beta).condition)$
  2. ...
4. Otherwise, $CP(\alpha)$ is $Partial(\alpha, CP(\beta_1) \wedge ... \wedge CP(\beta_n))$
 
**Reduction**: Reduction recurses into the body and the condition of a `Partial` just as if they were ordinary parts, after `CP`.  (In fact, I didn't need to change the reduction algorithm at all.)
* This implies, for example, that where $\beta$ does not contain $x$ free, $Partial(\lambda x . \alpha, \beta)$  and $\lambda x . Partial(\alpha, \beta)$ are equivalent under application with CP.


See partiality documentation notebook.

**reference**

Partial.calculate_partiality:
    
    def calculate_partiality(self):
        new_body = self.body.calculate_partiality()
        new_condition = self.condition.calculate_partiality()
        if isinstance(new_condition, Partial):
            new_condition = new_condition.body & new_condition.condition
        if isinstance(new_body, Partial):
            new_condition = new_condition & new_body.condition
            new_body = new_body.body
        new_condition = new_condition.simplify_all()
        return Partial(new_body, new_condition)

TypedExpr.calculate_partiality:

    def calculate_partiality(self):
        condition = true_term
        new_parts = list()
        for part in self:
            if isinstance(part, TypedExpr):
                part_i = part.calculate_partiality()
                if isinstance(part_i, Partial):
                    condition = condition & part_i.condition
                    part_i = part_i.body
            else:
                part_i = part
            new_parts.append(part_i)
        new_self = self.copy_local(*new_parts)
        condition = condition.simplify_all()
        if condition is not true_term:
            return Partial(new_self, condition)
        else:
            return self

LFun.calculate_partiality:

    def calculate_partiality(self):
        new_body = self.body.calculate_partiality()
        return self.copy_local(self.op, self.var_instance, new_body)


BindingOp.calculate_partiality is a bit more complicated.